In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'wisconsin', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: wisconsin
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the wisconsin graph
- Number of nodes: 251
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 21 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.74619 Time: 0.11804
Iteration: 0002 Loss: 1.63763 Time: 0.00296
Iteration: 0003 Loss: 1.53932 Time: 0.00304
Iteration: 0004 Loss: 1.57755 Time: 0.00401
Iteration: 0005 Loss: 1.49109 Time: 0.00399
Iteration: 0006 Loss: 1.61124 Time: 0.00396
Iteration: 0007 Loss: 1.55641 Time: 0.00306
Iteration: 0008 Loss: 1.54304 Time: 0.00399
Iteration: 0009 Loss: 1.52376 Time: 0.00400
Iteration: 0010 Loss: 1.47705 Time: 0.00407
Iteration: 0011 Lo

Iteration: 0199 Loss: 0.64592 Time: 0.00406
Iteration: 0200 Loss: 0.66051 Time: 0.00395
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 2 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 22 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.81158 Time: 0.08162
Iteration: 0002 Loss: 1.65810 Time: 0.00400
Iteration: 0003 Loss: 1.68281 Time: 0.00301
Iteration: 0004 Loss: 1.59383 Time: 0.00301
Iteration: 0005 Loss: 1.53572 Time: 0.00400
Iteration: 0006 L

Done! Louvain has found 20 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.62493 Time: 0.09499
Iteration: 0002 Loss: 1.66961 Time: 0.00601
Iteration: 0003 Loss: 1.65361 Time: 0.00499
Iteration: 0004 Loss: 1.67895 Time: 0.00301
Iteration: 0005 Loss: 1.50678 Time: 0.00399
Iteration: 0006 Loss: 1.67259 Time: 0.00300
Iteration: 0007 Loss: 1.58754 Time: 0.00300
Iteration: 0008 Loss: 1.62127 Time: 0.00400
Iteration: 0009 Loss: 1.61312 Time: 0.00400
Iteration: 0010 Loss: 1.53098 Time: 0.00400
Iteration: 0011 Loss: 1.55412 Time: 0.00300
Iteration: 0012 Loss: 1.47267 Time: 0.00401
Iteration: 0013 Loss: 1.53109 Time: 0.00299
Iteration: 0014 Loss: 1.41535 Time: 0.00300
Iteration: 0015 Loss: 1.43485 Time: 0.00301
Iteration: 0016 Loss: 1.41306 Time: 0.00305
Iteration: 0017 Loss: 1.52886 Time: 0.00400
Iteration: 0018 Loss: 1.44919 Time: 

Done! 

EXPERIMENTS ON MODEL 4 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 21 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.69639 Time: 0.10100
Iteration: 0002 Loss: 1.53420 Time: 0.00400
Iteration: 0003 Loss: 1.78776 Time: 0.00521
Iteration: 0004 Loss: 1.64417 Time: 0.00479
Iteration: 0005 Loss: 1.68520 Time: 0.00499
Iteration: 0006 Loss: 1.63230 Time: 0.00301
Iteration: 0007 Loss: 1.58250 Time: 0.00400
Iteration: 0008 Loss: 1.59965 Time: 0.00535
Iteration: 0009 Loss: 1.58913 Time: 0.00200
Iteration: 0010 Loss: 1.61724 Time: 0.00300
Iteration:

Iteration: 0196 Loss: 0.65902 Time: 0.00304
Iteration: 0197 Loss: 0.66846 Time: 0.00461
Iteration: 0198 Loss: 0.68034 Time: 0.00293
Iteration: 0199 Loss: 0.66646 Time: 0.00404
Iteration: 0200 Loss: 0.65382 Time: 0.00496
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 5 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 19 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.84294 Time: 0.10463
Iteration: 0002 Loss: 1.79830 Time: 0.00495
Iteration: 0003 L

...

Iteration: 0191 Loss: 0.67240 Time: 0.00400
Iteration: 0192 Loss: 0.67730 Time: 0.00295
Iteration: 0193 Loss: 0.65997 Time: 0.00299
Iteration: 0194 Loss: 0.65887 Time: 0.00403
Iteration: 0195 Loss: 0.66973 Time: 0.00300
Iteration: 0196 Loss: 0.65982 Time: 0.00435
Iteration: 0197 Loss: 0.67909 Time: 0.00561
Iteration: 0198 Loss: 0.66450 Time: 0.00404
Iteration: 0199 Loss: 0.66464 Time: 0.00297
Iteration: 0200 Loss: 0.66139 Time: 0.00410
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 7 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 21 communities 

Setting up the model and the optimizer
Done! 

Preprocessing

Iteration: 0185 Loss: 0.69200 Time: 0.00302
Iteration: 0186 Loss: 0.67477 Time: 0.00392
Iteration: 0187 Loss: 0.68063 Time: 0.00301
Iteration: 0188 Loss: 0.67507 Time: 0.00407
Iteration: 0189 Loss: 0.65989 Time: 0.00306
Iteration: 0190 Loss: 0.67543 Time: 0.00300
Iteration: 0191 Loss: 0.68951 Time: 0.00503
Iteration: 0192 Loss: 0.67184 Time: 0.00297
Iteration: 0193 Loss: 0.66833 Time: 0.00299
Iteration: 0194 Loss: 0.67915 Time: 0.00423
Iteration: 0195 Loss: 0.66121 Time: 0.00300
Iteration: 0196 Loss: 0.67109 Time: 0.00308
Iteration: 0197 Loss: 0.64711 Time: 0.00391
Iteration: 0198 Loss: 0.67299 Time: 0.00405
Iteration: 0199 Loss: 0.66494 Time: 0.00397
Iteration: 0200 Loss: 0.64985 Time: 0.00201
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 8 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - te

Iteration: 0185 Loss: 0.68114 Time: 0.00405
Iteration: 0186 Loss: 0.67403 Time: 0.00402
Iteration: 0187 Loss: 0.68597 Time: 0.00300
Iteration: 0188 Loss: 0.65675 Time: 0.00381
Iteration: 0189 Loss: 0.68856 Time: 0.00395
Iteration: 0190 Loss: 0.66519 Time: 0.00399
Iteration: 0191 Loss: 0.68140 Time: 0.00405
Iteration: 0192 Loss: 0.66461 Time: 0.00405
Iteration: 0193 Loss: 0.66928 Time: 0.00395
Iteration: 0194 Loss: 0.69348 Time: 0.00295
Iteration: 0195 Loss: 0.66157 Time: 0.00306
Iteration: 0196 Loss: 0.68450 Time: 0.00405
Iteration: 0197 Loss: 0.66049 Time: 0.00406
Iteration: 0198 Loss: 0.65770 Time: 0.00300
Iteration: 0199 Loss: 0.65599 Time: 0.00409
Iteration: 0200 Loss: 0.64630 Time: 0.00389
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 9 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - te

Iteration: 0197 Loss: 0.65711 Time: 0.00397
Iteration: 0198 Loss: 0.66241 Time: 0.00408
Iteration: 0199 Loss: 0.66301 Time: 0.00399
Iteration: 0200 Loss: 0.64391 Time: 0.00398
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 10 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 23 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.77960 Time: 0.15466
Iteration: 0002 Loss: 1.82481 Time: 0.00562
Iteration: 0003 Loss: 1.68307 Time: 0.00727
Iteration: 0004 

Iteration: 0195 Loss: 0.65109 Time: 0.00399
Iteration: 0196 Loss: 0.67401 Time: 0.00405
Iteration: 0197 Loss: 0.67855 Time: 0.00498
Iteration: 0198 Loss: 0.65919 Time: 0.00394
Iteration: 0199 Loss: 0.66709 Time: 0.00305
Iteration: 0200 Loss: 0.63248 Time: 0.00400
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 



In [5]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on wisconsin
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.6578765432098765
Std of AUC scores:  0.038040593115574145 

Mean AP score:  0.7437334045204061
Std of AP scores:  0.03917624111360606 
 

